# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
torch.cuda.is_available()

True

In [3]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [4]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [5]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [6]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [7]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [8]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [9]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [10]:
prompt_v1 = """### Task
You are an expert SQL generator. Your job is to carefully analyze the database schema and the question, then generate the **most accurate SQL query**.

### Reasoning Steps
1. Read the question carefully.
2. Identify which tables and columns are needed.
3. Join tables correctly using primary/foreign keys.
4. Apply filters based on dates, regions, or conditions.
5. Compute aggregates if necessary (SUM, AVG, COUNT, etc.).
6. Return a clean SQL query.

### Database Schema
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  price DECIMAL(10,2),
  quantity INTEGER
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY,
   name VARCHAR(50),
   address VARCHAR(100)
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY,
  name VARCHAR(50),
  region VARCHAR(50)
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  customer_id INTEGER,
  salesperson_id INTEGER,
  sale_date DATE,
  quantity INTEGER
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY,
  product_id INTEGER,
  supply_price DECIMAL(10,2)
);

-- Join notes:
-- sales.product_id -> products.product_id
-- sales.customer_id -> customers.customer_id
-- sales.salesperson_id -> salespeople.salesperson_id
-- product_suppliers.product_id -> products.product_id

### Answer
Write the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

prompt_v2 = """### Task
Generate a SQL query that answers the given business question using the database schema.

### Special Instructions
- "Revenue" = products.price * sales.quantity
- "Cost" = product_suppliers.supply_price * sales.quantity
- "Profit" = Revenue - Cost
- If the question asks for "by product" → group by product name
- If the question asks for "by region" → group by salesperson.region
- If you cannot answer with the schema, return "I do not know"

### Database Schema
(SAME AS BEFORE)

### Answer
Here is the SQL query to answer [QUESTION]{question}[/QUESTION]
[SQL]
"""

prompt_v3 = """You are a helpful data assistant. I will ask you business questions, and you will translate them into SQL queries that work on the provided database.

### Notes
- Always ensure your query runs without errors.
- Only use columns and tables that exist in the schema.
- Use aliases to make queries easy to read.
- Always return the result in a SELECT query (no explanations, just SQL).

### Database Schema
(SAME AS BEFORE)

### Question
{question}

### SQL Query
[SQL]
"""

In [18]:
def generate_query(question, prompt):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)


In [19]:
print("\n--- Prompt Version 1 ---")
print(generate_query(question, prompt_v1))

print("\n--- Prompt Version 2 ---")
print(generate_query(question, prompt_v2))

print("\n--- Prompt Version 3 ---")
print(generate_query(question, prompt_v3))



--- Prompt Version 1 ---

SELECT p.name,
       SUM(s.quantity) AS total_quantity
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.name;

--- Prompt Version 2 ---

SELECT p.name AS product_name,
       SUM(s.quantity * s.price) AS revenue
FROM sales s
JOIN products p ON s.product = p.id
JOIN salespeople sp ON s.salesperson = sp.id
WHERE sp.region = 'New York'
  AND EXTRACT(MONTH
              FROM s.date) = EXTRACT(MONTH
                                     FROM CURRENT_DATE - INTERVAL '1 month')
  AND EXTRACT(YEAR
              FROM s.date) = EXTRACT(YEAR
                                     FROM CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.name;

--- Prompt Version 3 ---

SELECT p.name,
       SUM(oi.quantity) AS total_quantity
FROM orders o
JOIN order_items oi ON o.id = oi.order_id
JOIN products p ON oi.product_id =

Report on SQL Generation with Transformer API

This report is based on the Jupyter notebook lab_sql_generation_with_transformer_api.ipynb. The main goal of the lab was to see how a transformer-based language model (specifically defog/sqlcoder-7b-2) can be used to turn normal questions into SQL queries. Basically, instead of writing the SQL ourselves, we give the model the question and some context, and it generates the query.

Setting Things Up

At the start, the notebook shows how to prepare the environment. This means installing the libraries we need like torch, transformers, bitsandbytes, and sqlparse. We also check if a GPU is available and how much memory it has. This is important because the model is pretty big. Depending on the GPU, the model is loaded either in float16 or 8-bit mode, which helps it fit better into memory.

How the Prompts Work

The main part of the lab was about creating a detailed prompt to help the model know what to do. The prompt includes the database schema, which has tables for products, customers, salespeople, sales, and product_suppliers. Each table has its own columns and data types.

The prompt also gives the model specific instructions, like:

How to calculate revenue and cost

What to return if the question can’t be answered

How to structure the SQL query

Example Question

One of the questions tested was:
“What was our revenue by product in the New York region last month?”

The model generated a pretty advanced SQL query. It joined the sales, salespeople, and products tables, filtered by the New York region and last month, and then grouped everything by product to calculate revenue.

What I Learned

Here are the main takeaways from the lab:

Prompts are super important. The results depend a lot on how clear and detailed the prompt is. If the prompt doesn’t define key terms (like revenue or profit) or misses context, the SQL might be wrong.

Prompt engineering really matters. The notebook tried three versions of prompts (v1, v2, v3), and each gave slightly different results. This shows how much wording and structure can change the output.

Models can make mistakes (hallucinations). Sometimes the model adds in fake tables or columns that don’t exist. This means we can’t just blindly trust its answers — we need to check the queries.

Hardware is a big deal. Running these big models needs a lot of GPU power. If your GPU doesn’t have enough memory, you need tricks like 8-bit loading just to make it work.